In [1]:
import akantu as aka
import numpy as np
from variationnal_operator_function import *

#### Mesh generation (from tutorial) :

In [2]:
# geometric parameters

w = 10.     # width (x-axis)
l = 5.      # length (y-axis)
h1 = 0.05   # characteristic mesh size at the hole
h2 = 0.3    # characteristic mesh size in corners
R = 2.      # radius of the hole

In [3]:
mesh_file = f"""
Point(1) = {{0, 0, 0, {h2} }};
Point(2) = {{ {R}, 0, 0, {h1} }};
Point(3) = {{ {w}, 0, 0, {h2} }};
Point(4) = {{ {w}, {l}, 0, {h2} }};
Point(5) = {{ 0,   {l}, 0, {h2} }};
Point(6) = {{0,    {R}, 0, {h1} }};
"""

In [4]:
mesh_file += """
Circle(1) = {6, 1, 2};
Line(2) = {2, 3};
Line(3) = {3, 4};
Line(4) = {4, 5};
Line(5) = {5, 6};
Line Loop(6) = {1, 2, 3, 4, 5};
Plane Surface(7) = {6};
"""

In [5]:
mesh_file += """
Physical Surface(8) = {7};
Physical Line("XBlocked") = {5};
Physical Line("YBlocked") = {2};
Physical Line("Traction") = {3};
"""

In [6]:
open('plate.geo', 'w').write(mesh_file)

import subprocess
ret = subprocess.run("gmsh -2 -order 1 -o plate.msh plate.geo", shell=True)
if ret.returncode:
    print("Beware, gmsh could not run: mesh is not regenerated")
else:
    print("Mesh generated")

Info    : Running '/opt/jlab-env/bin/gmsh -2 -order 1 -o plate.msh plate.geo' [Gmsh 4.11.1, 1 node, max. 1 thread]
Info    : Started on Tue Mar 12 08:43:22 2024
Info    : Reading 'plate.geo'...
Info    : Done reading 'plate.geo'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.0142238s, CPU 0.012701s)
Info    : Meshing 2D...
Info    : Meshing surface 7 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.108099s, CPU 0.087953s)
Info    : 2058 nodes 4118 elements
Info    : Writing 'plate.msh'...
Info    : Done writing 'plate.msh'
Info    : Stopped on Tue Mar 12 08:43:23 2024 (From start: Wall 0.417365s, CPU 0.619383s)
Mesh generated


In [7]:
# reading the mesh
spatial_dimension = 2    
mesh_file = 'plate.msh'
mesh = aka.Mesh(spatial_dimension)
mesh.read(mesh_file)

In [8]:
print(dir(mesh))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'addConnectivityType', 'addDumpField', 'addDumpFieldExternal', 'addDumpFieldExternalToDumper', 'addDumpFieldToDumper', 'addDumpMesh', 'addDumpMeshToDumper', 'createBoundaryGroupFromGeometry', 'createElementGroup', 'createElementGroupFromNodeGroup', 'createGroupsFromMeshDataUInt', 'createNodeGroup', 'distribute', 'dump', 'elementGroups', 'fillNodesToElements', 'getAssociatedElements', 'getConnectivities', 'getConnectivity', 'getDataReal', 'getElementGroup', 'getElementalDataReal', 'getElementalDataUInt', 'getLowerBounds', 'getMeshFacets', 'getNbElement', 'getNbNodes', 'getNodeGroup', 'getNodes', 'getPeriodicMaster', 'getPeriodicSlaves', 'getSpatialDimension',

#### Support declaration

In [9]:
model = aka.SolidMechanicsModel(mesh)

In [10]:
elem_filter = np.array([])
fem = model.getFEEngine()
spatial_dimension = 2
elemtype = aka._triangle_3
ghost_type = aka.GhostType(1) #comment avoir ghost_type vide?
Sup = Support(elem_filter, fem, spatial_dimension, elemtype, ghost_type)

#### Interpolation

In [11]:
# create a field  
nodes = mesh.getNodes()
nodal_field=np.ones(nodes.shape)*3

In [12]:
NTF = NodalTensorField("test", Sup, nodal_field)

In [14]:
output = aka.ElementTypeMapArrayReal()
output.initialize(mesh, nb_component=2)
NTF.evalOnQuadraturePoints(output)

TypeError: interpolateOnIntegrationPoints(): incompatible function arguments. The following argument types are supported:
    1. (self: akantu.py11_akantu.FEEngine, u: AkaArray, uq: AkaArray, nb_degree_of_freedom: int, type: akantu.py11_akantu.ElementType, ghost_type: akantu.py11_akantu.GhostType = <GhostType._not_ghost: 0>, filter_elements: AkaArray = array([], shape=(0, 1), dtype=int32)) -> None
    2. (self: akantu.py11_akantu.FEEngine, u: AkaArray, uq: akantu.py11_akantu.ElementTypeMapArrayReal, filter_elements: akantu.py11_akantu.ElementTypeMapArrayInt = None) -> None

Invoked with: <akantu.py11_akantu.FEEngine object at 0x7f9be1aa2730>, array([[3., 3.],
       [3., 3.],
       [3., 3.],
       ...,
       [3., 3.],
       [3., 3.],
       [3., 3.]]), <akantu.py11_akantu.ElementTypeMapArrayReal object at 0x7f9be1ab65b0>, 4114, <ElementType._triangle_3: 4>